DATASET LOADING

In [ ]:
import pandas as pd
import numpy as np

dataset = pd.read_csv(PATH)
df_raw = pd.DataFrame(dataset)

#filter predictors
df = df_raw[['X','idade','genero','fc','fr','pa_sist','pa_diast','pam','temp','hb',
              'plaquetas','ht','hemacias','hcm','rdw','vcm','leucocitos','neutrofilos',
              'linfocitos','basofilos','eosinofilos','monocitos','proteina_c_reativa', "obito"]]

#filter patients by age and PCR
df = df.loc[df['rt_pcr_covid'] == "Positivo"]
df = df[df["idade"] >= 18]

#one hot encoding
df["genero"] = np.select([df["genero"] == "Masc", df["genero"] == "Fem"], [0, 1])
df['obito'] = df['obito']. fillna(0)
df['obito'] = df['obito'].apply(lambda x: 1 if x != 0 else 0)


EXPLORATORY ANALYSIS

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df.shape
df.head()
df.info()

df[numeric_features].describe().T

for feature in dummy_features:
  print(df[feature].value_counts(normalize = True))

#boxplots
plt.figure(figsize =(30,100))

for i, feature in enumerate(numeric_features, 1):
  plt.subplot(10,2,i)
  sns.boxplot(x=df[feature])
  plt.title(f'Boxplot of {feature}')

plt.tight_layout()
plt.show()

#histograms
plt.figure(figsize =(30,100))

for i, feature in enumerate(numeric_features, 1):
  plt.subplot(10,2,i)
  sns.histplot(x=df[feature])
  plt.title(f'Histogram of {feature}')

plt.tight_layout()
plt.show()

DATA PREPROCESSING

In [ ]:
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

#missing values
df.isnull().any()
msno.matrix(df)

df['missing_exame_sangue'] = 0
df.loc[(df['missing_exame_sangue'] == '') | (df['hb'].isna()), 'missing_exame_sangue'] = 1
df['missing_exame_sangue'].value_counts(normalize=True)


#train-test split
X_df = df.drop(columns=["obito"])
y_df = df[["obito"]]

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3, random_state=42, stratify= y_df)

X_train[numeric_features].describe().T

#missing filter and imputation
for column in X_train.columns:
  if  X_train[column].isnull().mean() >= 0.9:
    X_train = X_train.drop(column, axis=1)
    X_test = X_test.drop(column, axis =1)
  else:
    X_train[column] = X_train[column].fillna(X_train[column].median())
    X_test [column] = X_test [column].fillna(X_train[column].median())

#correlation filter
corr = X_train[numeric_features].corr()
for i in range(len(numeric_features)):
  for j in range(len(numeric_features)):
    if i!=j and corr.iloc[i, j] > 0.9:
      X_train = X_train.drop(X_train.columns[i], axis=1)
      X_test = X_test.drop(X_train.columns[i], axis=1)

#normalization
scaler = StandardScaler()
scaler = scaler.fit(X_train[numeric_features])
X_train[numeric_features] = scaler.transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])

#class balancing
oversample = RandomOverSampler(sampling_strategy='minority', random_state =42)
X_train, y_train = oversample.fit_resample(X_train, y_train)


MODEL TRAINNING

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import shap

kf = KFold(n_splits=10, shuffle=True, random_state=42)

def objective(params):
    auc_scores = []

    for train_index, val_index in kf.split(X_train):
      x_t, x_v = X_train[train_index], X_train[val_index]
      y_t, y_v = y_train[train_index], y_train[val_index]

      clf.fit(x_t, y_t)
      preds = clf.predict_proba(x_v)[:, 1]
      auc = roc_auc_score(y_v, preds)
      auc_scores.append(auc)

    auc = np.mean(auc_scores)
    return {'loss': -auc, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn=objective,
        space=param_space,
        algo=tpe.suggest,
        max_evals=100,
        trials=trials)

final_model = clf(**best, use_label_encoder=False, eval_metric='auc')
final_model.fit(X_train, y_train)

preds = final_model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, preds)
RocCurveDisplay.from_predictions(y_test, preds)

preds = [0 if x <0.5 else 1 for x in preds]
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)

cf_matrix = confusion_matrix(y_test, preds)
cf_percent = cf_matrix.astype('float') / cf_matrix.sum()
fig, ax = plt.subplots(figsize=(8, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cf_percent, display_labels=final_model.classes_)
disp.plot(cmap=plt.cm.Blues, ax=ax, values_format=".2f")

misclassified = []
for index in range(len(y_test)):
    if int(y_test.iloc[index]) != preds[index]:
       misclassified.append(index)

explainer = shap.Explainer(final_model)
shap_values = explainer.shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="dot")

#TabPFN
from tabpfn import TabPFNClassifier
from sklearn.metrics import precision_recall_curve
import shapiq

tabpfn_model = TabPFNClassifier()

tabpfn_model.fit(X_train, y_train)
preds = tabpfn_model.predict_proba(X_test)

auc = roc_auc_score(y_test, (preds[:, 1]))
RocCurveDisplay.from_predictions(y_test, (preds[:, 1]))

precision, recall, thresholds = precision_recall_curve(y_test, preds[:, 1])
f1_scores = 2 * (precision * recall) / (precision + recall)
best_threshold = thresholds[f1_scores.argmax()]
tab_final_preds = (tab_final_preds[:, 1] > best_threshold).astype(int)

tab_accuracy = accuracy_score(y_test, tab_final_preds)
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)

cf_matrix = confusion_matrix(y_test, preds)
cf_percent = cf_matrix.astype('float') / cf_matrix.sum()
fig, ax = plt.subplots(figsize=(8, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cf_percent, display_labels=final_model.classes_)
disp.plot(cmap=plt.cm.Blues, ax=ax, values_format=".2f")

misclassified = []
for index in range(len(y_test)):
    if int(y_test.iloc[index]) != preds[index]:
       misclassified.append(index)

x_explain = X_test.values[200]
y_explain = y_test.values[200]

explainer = shapiq.Explainer(
    model=TabPFNClassifier(device = 'cpu'),
    data=X_train_subset.values,
    labels=y_train_subset,
    index="FSII",
    max_order=1,
)
explainer._imputer.verbose = True
fsii = explainer.explain(x_explain, budget=100)
display(fsii.dict_values)
fsii.plot_force(feature_names=feature_names)

PROBABILISTIC PREDICTIONS CORRELATION

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.colors as mcolors

df_long = df.reset_index().rename(columns={'index': 'hospital'})
model_cols = ['xgb_preds', 'rf_preds', 'lgb_preds', 'cat_preds', 'tab_preds']
long_format = []

for _, row in df_long.iterrows():
    hospital = row['hospital']
    preds_dict = {model: row[model] for model in model_cols}
    num_pacientes = len(row[model_cols[0]])

    temp_df = pd.DataFrame({
        'hospital': [hospital] * num_pacientes,
        **{model: preds_dict[model] for model in model_cols}
    })

    long_format.append(temp_df)

df_final = pd.concat(long_format, ignore_index=True)

sns.set_style('white')
palette = sns.color_palette("YlGnBu", 5)

g = sns.pairplot(df_final, hue='Hospital', aspect=1, diag_kind= 'hist', corner=True, palette =palette)

for ax in g.axes.flat:
    if ax:
        ax.set_xlim(-0.02, 1.02)
        ax.set_ylim(-0.02, 1.02)

vars = df_final.columns[1:]

for i in range(len(vars)):
    for j in range(i):
        x = df_final[vars[j]].values.reshape(-1, 1)
        y = df_final[vars[i]].values
        model = LinearRegression().fit(x, y)
        r2 = r2_score(y, model.predict(x))

        ax = g.axes[i, j]
        x_plot = x.flatten()
        y_pred = model.predict(x)
        ax.plot(x_plot, y_pred, color='blue')

        ax.set_xlabel(ax.get_xlabel(), fontsize=13.5, labelpad=10)
        ax.set_ylabel(ax.get_ylabel(), fontsize=13.5, labelpad=10)

        ax.annotate(f"$R^2$ = {r2:.2f}",
                    xy=(0.03, 0.96), xycoords='axes fraction',
                    ha='left', va='top', fontsize=14, color='black',
                    bbox=dict(boxstyle='round,pad=0.2', facecolor='white', edgecolor='none', alpha=0.8))


g._legend.set_title('Hospital')
for text in g._legend.texts:
    text.set_fontsize(13)

title = g._legend.get_title()
title.set_fontsize(14)
title.set_fontweight('bold')

g._legend.set_bbox_to_anchor((0.8, 0.75))
g._legend.set_frame_on(True)

plt.tight_layout()

PROFILE GRAPH BY ERROR GROUP

In [ ]:
mean_vectors = pd.DataFrame()

for algorithm in algorithms:
  X_test_sampled = X_test.loc[list(df_errors[algorithm].dropna())]
  mean_vector = X_test_sampled.mean()
  mean_vectors[algorithm] = mean_vector

flat_misclassified = [item for sublist in misclassified for item in sublist]
X_test_not_misclassified = X_test.loc[list(set(X_test.index) - set(flat_misclassified))]

mean_vectors["Not misclassified"] = X_test_not_misclassified.mean()

fig, ax = plt.subplots(figsize=(10, 6))
colors = sns.color_palette("YlGnBu", n_colors=len(legend))

for i, (algorithm, mean_vector) in enumerate(mean_vectors.items()):
    ax.plot(mean_vector.index, mean_vector.values, linestyle='--', marker= ".", markersize = 12, label=legend[algorithm], color = colors[i])

plt.axhline(y=0, color='grey')
plt.grid(axis='x', linestyle='--', alpha=0.7)
translated_labels = [x_label.get(label, label) for label in mean_vector.index]
ax.set_xticks(range(len(translated_labels)))
ax.set_xticklabels(translated_labels, rotation=90)
ax.set_xlabel('Variables')
ax.set_ylabel('Mean')

ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()

STATISTICAL TESTS

In [ ]:
import scipy
from scipy import stats
from scipy.stats import mannwhitneyu

all_patients = [i for i in range(len(X_test))]

for x in all_patients:
  for model in models:
    if x in misclassified[model]:
        matriz[x][models.index(model)] = 1

df_errors = pd.DataFrame.from_dict(matriz, orient='index', columns=['Patient', 'XGBoost', 'Random Forest', 'TabPFN', 'LightGBM', 'CatBoost'])
target = ['XGBoost', 'Random Forest', 'TabPFN', 'LightGBM', 'CatBoost']
error_all_models = []
no_error = []

for index, row in df_errors.iterrows():
    soma = row[target].sum()
    if soma == 0:
      no_error.append(row["Patient"])
    elif soma == 5:
      error_all_models.append(row["Patient"])

sampled_indexes = no_error + error_all_models
X_test_sampled = X_test.loc[sampled_indexes]
X_test_sampled['error'] = X_test_sampled.index.map(lambda i: 1 if i in error_all_models else 0)

data_1 = X_test_sampled.loc[X_test_sampled['error'] ==1]
data_0 = X_test_sampled.loc[X_test_sampled['error'] ==0]

#Shapiro-Wilk, normal distribution
for feature in X_test_sampled:
  s = scipy.stats.shapiro(X_test_sampled[feature])
  if s.pvalue < 0.005:
    not_normal.append(feature)
  else:
    normal.append(feature)

#Levene, homocedasticity
for i, feature in enumerate(X_test_sampled.columns[:-1], 1):
  s = stats.levene(data_0[feature], data_1[feature], center= 'mean')
  if s.pvalue < 0.005:
    diff_var.append(feature)
  else:
    homocedastic.append(feature)

#Paired t-test
t_features = set(normal).intersection(homocedastic)
for i, feature in enumerate(t_features, 1):
  t_stat, p_value = stats.ttest_ind(data_0[feature], data_1[feature])
  if p_value < 0.005:
    reject.append(feature)

#Mann-Whitney U test
mw_features = set(X_test_sampled.columns[:-1]).difference(t_features)
for i, feature in enumerate(mw_features, 1):
  test_stat, p_value = mannwhitneyu(data_0[feature], data_1[feature])
  if p_value < 0.005:
    reject.append(feature)

SHAP BOXPLOTS

In [ ]:
shap_data = pd.DataFrame()
for hospital in hospitals:
  for algorithm in algorithms:
    path = f"/content/drive/MyDrive/{hospital}/{algorithm}_shap_values{hospital}.csv"
    df = pd.read_csv(path, sep =";")
    for column in df.columns:
      df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0).abs()
    shap_data = pd.concat([shap_data, df], ignore_index=True)

shap_data = shap_data.drop("Unnamed: 0", axis=1)
mean_shap = shap_data.mean().sort_values(ascending=False)

top_features = mean_shap.head(10).index
shap_top = shap_data[top_features].melt(var_name="Feature", value_name="SHAP Value")

reversed_blues = sns.color_palette("Blues", n_colors=10, as_cmap=False)[::-1]

plt.figure(figsize=(10, 6))
ax = sns.boxplot(data=shap_top, x="Feature", y="SHAP Value", palette=reversed_blues, showfliers=False)

translated_labels = [x_label.get(label, label) for label in  shap_top["Feature"].unique()]

ax.set_xticks(range(len(translated_labels)))
ax.set_xticklabels(translated_labels, rotation=45, ha="right")
plt.xlabel("")
plt.ylabel("SHAP Value")
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.show()

CLUSTER ANALYSIS

In [ ]:
from sklearn.cluster import KMeans

def calculate_wcss(data):
    wcss = []
    for n in range(2, 11):
        kmeans = KMeans(n_clusters=n, random_state=42)
        kmeans.fit(X=data)
        wcss.append(kmeans.inertia_)
    return wcss

def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 11, wcss[len(wcss)-1]
    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    return distances.index(max(distances)) + 2

sum_of_squares = calculate_wcss(X_train)
n = optimal_number_of_clusters(sum_of_squares)

plt.axvline(n)
plt.plot(range(2,11), sum_of_squares)
plt.xlabel("Number of clusters")
plt.ylabel("WCSS")
plt.show()

kmeans = KMeans(n_clusters=n, random_state=43)
clusters = kmeans.fit_predict(X_train)

translated_features = [x_label[feat] for feat in top_features]

X_train_sampled = X_train[top_features]
X_train_sampled['cluster'] = clusters

mean_vectors = X_train_sampled.groupby('cluster').mean()
mean_vectors.rename(columns=x_label, inplace=True)

n = mean_vectors.shape[0]
fig, axes = plt.subplots(nrows=1, ncols=n, figsize=(24, 12), subplot_kw={'projection': 'polar'})

features = mean_vectors.columns
num_features = len(features)
angles = np.linspace(0, 2 * np.pi, num_features, endpoint=False).tolist()

for i, (cluster_id, mean_vector) in enumerate(mean_vectors.iterrows()):
    ax = axes[i]
    values = mean_vector.values.tolist()
    angles_closed = angles + angles[:1]
    values_closed = values + values[:1]

    ax.plot(angles_closed, values_closed, label=f"{cluster_names[cluster_id]}", linewidth=2)
    ax.fill(angles_closed, values_closed, alpha=0.25)

    ax.set_yticklabels([])
    ax.set_xticks(angles_closed)

    feature_labels = features.tolist() + [features[0]]
    ax.set_xticklabels(feature_labels, rotation=45, ha='right')
    ax.set_title(f'{cluster_names[cluster_id]}', size=14, y=1.25)

plt.tight_layout()
plt.show()

df_train = pd.concat([X_train, y_train], axis=1)
df_train.groupby("cluster")["obito"].mean()

#Trainning with 10-fold cross validation
for cluster in cluster_list:
    data = df_train.loc[df_train['cluster'] == cluster]
    X = data.drop(['obito', 'cluster'], axis=1)
    y = data['obito'].fillna(0)

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=43)

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # XGBoost
        xgb_model = xgb.XGBClassifier()
        xgb_model.fit(x_train, y_train)
        xgb_pred = xgb_model.predict_proba(x_test)[:, 1]
        xgb_auc = roc_auc_score(y_test, xgb_pred)

        # Random Forest
        rf_model = RandomForestClassifier()
        rf_model.fit(x_train, y_train)
        rf_pred = rf_model.predict_proba(x_test)[:, 1]
        rf_auc = roc_auc_score(y_test, rf_pred)

        # CatBoost
        cat_model = CatBoostClassifier(verbose=0)
        cat_model.fit(x_train, y_train)
        cat_pred = cat_model.predict_proba(x_test)[:, 1]
        cat_auc = roc_auc_score(y_test, cat_pred)

        # LightGBM
        lgb_model = lgb.LGBMClassifier()
        lgb_model.fit(x_train, y_train)
        lgb_pred = lgb_model.predict_proba(x_test)[:, 1]
        lgb_auc = roc_auc_score(y_test, lgb_pred)

        # TabPFN
        tab_model = TabPFNClassifier(device='cpu')
        tab_model.fit(x_train, y_train)
        tab_pred = tab_model.predict_proba(x_test)[:, 1]
        tab_auc = roc_auc_score(y_test, tab_pred)

        # AUCs
        auc_scores = pd.concat([
            auc_scores,
            pd.DataFrame([{
                'cluster': cluster,
                'fold': fold,
                'xgb': xgb_auc,
                'rf': rf_auc,
                'cat': cat_auc,
                'lgb': lgb_auc,
                'tab': tab_auc
            }])
        ], ignore_index=True)


#Performance comparison graph
data_means = []
data_errors = []

for cluster in range(0, 5):
  data = df[df["cluster"] == cluster]

  list_means = []
  mean_xgb = float(np.mean(data["xgb"]))
  mean_cat = float(np.mean(data["cat"]))
  mean_lgb = float(np.mean(data["lgb"]))
  mean_rf = float(np.mean(data["rf"]))
  mean_tab = float(np.mean(data["tab"]))
  list_means = [mean_xgb, mean_cat, mean_lgb, mean_rf, mean_tab]

  data_means.append(list_means)

  std_xgb = np.std(data["xgb"])
  std_cat = np.std(data["cat"])
  std_lgb = np.std(data["lgb"])
  std_rf = np.std(data["rf"])
  std_tab = np.std(data["tab"])

  erro_padrao_xgb = std_xgb / np.sqrt(len(data["xgb"]))
  erro_padrao_cat = std_cat / np.sqrt(len(data["cat"]))
  erro_padrao_lgb = std_lgb / np.sqrt(len(data["lgb"]))
  erro_padrao_rf = std_rf / np.sqrt(len(data["rf"]))
  erro_padrao_tab = std_tab / np.sqrt(len(data["tab"]))

  # degrees of freedom
  g = 9

  list_interval = []
  xgb_interval = st.t.interval(confidence=0.95, df=g, loc=mean_xgb, scale=erro_padrao_xgb)
  cat_interval = st.t.interval(confidence=0.95, df=g, loc=mean_cat, scale=erro_padrao_cat)
  lgb_interval = st.t.interval(confidence=0.95, df=g, loc=mean_lgb, scale=erro_padrao_lgb)
  rf_interval = st.t.interval(confidence=0.95, df=g, loc=mean_rf, scale=erro_padrao_rf)
  tab_interval = st.t.interval(confidence=0.95, df=g, loc=mean_tab, scale=erro_padrao_tab)
  print(tab_interval)

  errors = [float(mean - low) for mean, low in zip(list_means, [xgb_interval[0], cat_interval[0], lgb_interval[0], rf_interval[0], tab_interval[0]])]

  data_errors.append(errors)

clusters = ["Anaemia","Young", "Immune Activation", "Reccovery", "Immunodeficient"]
alg = ["XGBoost", "Random Forest", "Catboost", "LightGBM", "TabPFN"]
data = data_means

data = np.array(data).T
x = np.arange(len(alg))
width = 0.13

fig, ax = plt.subplots(figsize=(12, 8))
ax.grid(axis="y", linewidth=0.5, ls ="--")
ax.set_axisbelow(True)

colors = sns.color_palette("YlGnBu")

for i in range(len(alg)):
    ax.bar(x + i*width - width*2, data[i], width, label=alg[i], color=colors[i], yerr=errors[i], capsize= 4, ecolor=colors[i+1])

ax.set_ylabel('Score')
ax.set_ylim(0, 1)
ax.set_xticks(x)
ax.set_xticklabels(clusters, rotation=0)
ax.set_yticks(np.arange(0, 1.05, 0.05))
ax.legend(loc='upper right')
ax.set_ylim(0.5, 1.02)

plt.tight_layout()